# 01. OCR을 활용한 문자 인식

## 01-1. OCR 개요

### OCR이란?
OCR(Optical Character Recognition, 광학 문자 인식)은 이미지 내의 텍스트를 감지하고 이를 디지털 텍스트로 변환하는 기술입니다. OCR은 문서 자동화, 차량 번호판 인식, 전자상거래 및 금융 서비스에서 널리 사용됩니다.

### OCR의 활용 사례
- **문서 스캔 및 디지털화**: 문서를 스캔하여 편집 가능한 형태로 변환합니다.
- **자동차 번호판 인식**: 교통 감시 및 주차 관리 시스템에서 사용됩니다.
- **바코드 및 영수증 처리**: 쇼핑몰 및 물류 관리 시스템에서 활용됩니다.
- **자연어 처리(NLP)와 결합**: OCR을 통해 추출한 텍스트를 분석하여 의미를 파악합니다.

## 01-2. OCR 동작 원리
### 전처리 단계
- **이진화(Binarization)**: 이미지의 배경과 문자를 분리하기 위해 그레이스케일 변환 후 Otsu’s Thresholding을 적용합니다.
- **노이즈 제거**: 가우시안 블러(Gaussian Blur) 및 모폴로지 연산을 활용하여 배경을 제거합니다.
- **문자 정렬 및 기울기 보정**: Hough Transform을 활용하여 텍스트를 정렬합니다.

### 문자 인식 과정
- **세그멘테이션(Segmentation)**: 문자를 개별적으로 분할합니다.
- **피처 추출(Feature Extraction)**: 문자 패턴을 추출하여 벡터화합니다.
- **딥러닝 기반 인식**: CNN과 RNN을 활용하여 문자를 분류하고 해석합니다.

## 01-3. 주요 OCR 기술
### 전통적인 OCR 엔진
- **Tesseract OCR** (Google 지원): 오픈소스로 제공되는 강력한 OCR 엔진입니다.
  - LSTM 기반의 문자 인식을 지원합니다.
  - 다양한 언어를 지원하며, 사용자 정의 학습이 가능합니다.
  
### 딥러닝 기반 OCR
- **CRNN (Convolutional Recurrent Neural Network)**
  - CNN을 통해 특징을 추출하고 RNN을 활용하여 문맥을 이해하여 OCR 성능을 향상합니다.
- **EAST (Efficient and Accurate Scene Text Detector)**
  - 실시간 텍스트 검출이 가능하며, OpenCV와 TensorFlow에서 지원됩니다.

---

# 02. [실습 9] 라즈베리파이 OCR 활용

## 02-1. OpenCV + Tesseract

### 환경 설정
```bash
pip install opencv-python pytesseract
sudo apt install tesseract-ocr

# 한글 인식을 원할 경우
sudo apt install tesseract-ocr-kor
```

### OpenCV를 활용한 이미지 전처리
이미지에서 텍스트를 보다 잘 추출하기 위해 OpenCV를 활용하여 대비 조정 및 이진화를 적용한다.

In [17]:
import cv2
import pytesseract

# 이미지 로드
image = cv2.imread('sample_text.jpg')

# 그레이스케일 변환
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 이진화 처리
thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)[1]

# OCR 수행
text = pytesseract.image_to_string(thresh, lang='eng')
print("인식된 텍스트:")
print(text)

인식된 텍스트:
JAVA Ol)

JESUS PAID IT ALL

SIN PAID
SHAME map ae)
REGRET mapa
al UP ed Ee 4 t-) ep ae)
UNF ORG I VENESS mPa
HURT PAID
ANGER PAID

Subtotal

nt 5 60

PS ee Le 2 2 22 101017, meeearee

Pod

"AND THEY SAID,
BELIEVE ON THE LORD JESUS CHRIST,
' AND THOU SHALT BE SAVED,
AND THY HOUSE."
ACTS 16:31

LT



## 02-2. OpenCV + YOLO

### YOLO 모델 로드

```bash
wget https://github.com/Muhammad-Zeerak-Khan/Automatic-License-Plate-Recognition-using-YOLOv8/raw/main/license_plate_detector.pt
```

In [18]:
import cv2
import torch
from ultralytics import YOLO
import pytesseract

# 학습된 YOLOv8 모델 로드
model = YOLO('license_plate_detector.pt')

# 이미지 불러오기
image_path = 'sample_car.jpg'  # 분석할 이미지 경로
image = cv2.imread(image_path)

# YOLOv8을 사용하여 번호판 검출
results = model(image)

# 검출된 번호판에 대한 OCR 수행
for result in results:
    boxes = result.boxes.xyxy  # 바운딩 박스 좌표
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)

        # 번호판 영역 추출
        plate_image = image[y1:y2, x1:x2]

        # OCR 전처리: 그레이스케일 변환 및 이진화
        gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # OCR 수행
        text = pytesseract.image_to_string(thresh, config='--psm 7')
        print(f'인식된 번호판: {text.strip()}')

        # 결과 시각화
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, text.strip(), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# 결과 이미지 표시
cv2.imshow('License Plate Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 448x640 1 license_plate, 2.6ms
Speed: 1.4ms preprocess, 2.6ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)
인식된 번호판: 


In [23]:
import cv2
import torch
import pytesseract
from ultralytics import YOLO

# YOLOv8 번호판 검출 모델 로드
model = YOLO("license_plate_detector.pt")

# 이미지 로드
image_path = "sample_car.jpg"  # 분석할 이미지 경로
image = cv2.imread(image_path)

if image is None:
    raise FileNotFoundError(f"🚨 오류: 이미지 파일 '{image_path}'을 찾을 수 없습니다.")

# YOLO를 사용하여 번호판 탐지
results = model(image)

# 번호판 검출 및 OCR 수행
for result in results:
    boxes = result.boxes.xyxy  # 바운딩 박스 좌표 (x1, y1, x2, y2)
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)

        # 번호판 영역(ROI) 잘라내기
        roi = image[y1:y2, x1:x2]

        # ROI 확인 (번호판이 잘 잘렸는지 보기 위해 출력)
        cv2.imshow(f"License Plate ROI {i}", roi)  # 개별 번호판 확인
        cv2.imwrite(f"license_plate_roi_{i}.jpg", roi)  # 파일로 저장

        # OCR 전처리
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # OCR 수행
        text = pytesseract.image_to_string(thresh, config="--psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789가나다라마바사아자차카타파하")
        text = text.strip()

        print(f"🔍 인식된 번호판 {i}: {text}")

        # 검출된 번호판 시각화
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# 결과 이미지 출력
cv2.imshow("License Plate Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 448x640 1 license_plate, 3.3ms
Speed: 1.5ms preprocess, 3.3ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)
🔍 인식된 번호판 0: 
